In [1]:
from PIL import ImageFont, ImageDraw, Image
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
!pip install opencv-python
import cv2 as cv

%matplotlib inline

In [2]:
def max_contrast(img):
    
    """
    
        maximize contrast of images
        also deleting "edges"
        
    """
    
    img[img == img[0,0]] = 0
    
    mi,ma = img.min(),img.max()
    if ma == 0:
        return img
    
    img -= mi
    img[img == ma -mi] = 0
    ma = img.max()
    img = np.array(((img / ma) * 255),dtype='uint8')
    return img

def binary(img,threshold=5):
    img[img > threshold ] = 255
    img[img <= threshold] = 0
    return img


In [3]:
path = "PNG"

"""

        Plot difference between two images

"""

fig, axes = plt.subplots(1, 3, figsize=(10, 10),dpi=100, sharex=False, sharey=True)
img_0 = Image.open(os.path.join(path,"scaled_1705140100.png"))
img_1 = Image.open(os.path.join(path,"scaled_1705140105.png"))
img_0 = max_contrast(np.array(img_0))
img_0 = np.array(img_0)
img_1 = max_contrast(np.array(img_1))
img_0 = binary(img_0)
img_1 = binary(img_1)
axes[0].imshow(img_0,cmap="gray")
axes[1].imshow(img_1,cmap="gray")
axes[2].imshow(np.abs(img_0-img_1),cmap="gray")
plt.show()

In [4]:
def get_files(pwd,pattern=""):
    
    """
    
        Read all files from path which match a pattern and return a list with the names
    
    """
    
    import re
    files = []
    for file in os.listdir(pwd):
        matches = re.search(pattern, file)
        if matches:
            files.append(matches[0])
    files.sort()
    return files

files = get_files(path,pattern="scaled_17051.*")

def show_series(function):
    
    """
        
        shows a series of images,
        function should return images
    
    """
    
    windowname = 'OpenCvFrame'
    cv.namedWindow(windowname)
    cv.moveWindow(windowname,2600,40)
    ret = True

    while(ret):
        for file in files:
            frame = function(file)
            cv.imshow(windowname, frame)

            if cv.waitKey(25) & 0XFF == ord('q'):
                ret = False
                break


    cv.destroyAllWindows()


In [5]:
scale = 30
def create_GIF(function,name="clouds.gif",nbr_imgs = 0):

    
    def create(inputPath, outputPath, delay, finalDelay, loop):
        cmd = "convert -delay {} {}*.png -delay {} -loop {} {}".format(
        delay, inputPath, finalDelay, loop,
        outputPath)
        os.system(cmd)

    folder = "GIF/"

    if not os.path.exists(folder):
        os.mkdir(folder)
    files.sort()
    for i,file in enumerate(files):
        original = Image.open(os.path.join(path,file))
        original = np.array(original)
        original[original != 0] = 255
        img = function(file)
        original = cv.cvtColor(original,cv.COLOR_GRAY2RGB)
        
        concat = np.concatenate((original,img),axis=1)
        cv.imwrite(os.path.join(folder,file),concat)
        if i == nbr_imgs and nbr_imgs != 0:
            break

    create(folder,name,10,250,0)
    



In [10]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

class MplColorHelper:

    def __init__(self, cmap_name, start_val, stop_val):
        if stop_val > 255:
            stop_val = 255
        self.cmap_name = cmap_name
        self.cmap = plt.get_cmap(cmap_name)
        self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
        self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

    def get_rgb(self, val):
        rgb = list(self.scalarMap.to_rgba(val))[:3]
        ret = [int(i * 255) for i in rgb]
        #return rgb
        return [int(rgb[0]*255),int(rgb[1]*255),int(rgb[2]*255)]


def sequentialLabeling(img,max_dist=1):

    def consecutive(data, stepsize=1):
        return np.split(data, np.where(np.diff(data) == stepsize)[0]+1)

    img[img >= 5] = 1
    x,y = np.where(img == 1)

    
    collision = dict()
    label = 2

    for i,j in zip(x,y):
        i_X = slice(i-max_dist,i+max_dist)
        j_Y = slice(j-max_dist,j+max_dist)
        
        window = img[i_X,j_Y]
        
        neighbours = np.argwhere(window > 1)
        
        
        if len(neighbours) == 0:
            window[window == 1] = label
            label +=1
            img[i_X,j_Y] = window
            
        elif len(neighbours) == 1:
            window[window == 1] = window[neighbours[0,0],neighbours[0,1]]
            img[i_X,j_Y] = window
        
        
        # handle label collisions
        
        else:
            k = np.amax(window)
            img[i,j] = k
            for index in neighbours:
                nj = window[index[0], index[1]]
                        
                if nj != k:
                    if k not in collision:
                        collision[k] = set()
                    collision[k].add(nj)
                    if collision[k] is None:
                        del collision[k]

    
    
    def changeLabel(elem):
        c_label = collision[elem]
        for l in c_label:
            img[img == l] = elem
        

    def rearangeCollisions():
        for elem in collision:
            for item in collision[elem]:
                if item in collision:
                    collision[elem] = (collision[elem] | collision[item])
                    collision[item] = set()
            if elem in collision[elem]:
                collision[elem].remove(elem)
            
    rearangeCollisions()
    
    
    for i,elem in enumerate(collision):
        if collision[elem] is None:
            continue
        changeLabel(elem)

    # paint clouds according to their size
    
    cloud_size = []
    
    for i in range(2,label):
        a = len(np.where(img == i)[0])
        if a == 0:
            continue
        cloud_size.append((i,a))
    cloud_size = sorted(cloud_size, key=lambda x: x[1],reverse = True)
    
    return cloud_size

def mapToColor(img,cloud_size):
    COL = MplColorHelper('hsv', 0, len(cloud_size))

    colors = {}
    for i,elem in enumerate(cloud_size):
        colors[elem[1]] = COL.get_rgb(i)
    img = cv.cvtColor(img,cv.COLOR_GRAY2RGB)

    for label,size in cloud_size:
        index = np.where(img[:,:,0] == label)
        img[index] = colors[size]

    return img


def computeShell(img,label):
    sob = cv.Canny(img,130,130)
    #sob = cv.cvtColor(sob,cv.COLOR_GRAY2RGB)
    return sob
    for l in label:
        indizes = np.where(img == l[0])
        print(indizes)


        
def addText(img,coord,text):
    
        font                   = cv.FONT_HERSHEY_COMPLEX_SMALL
        fontScale              = 0.5
        fontColor              = (255,255,255)
        lineType               = 1
    
        cv.putText(img,
            text, 
            coord,
            font, 
            fontScale,
            fontColor,
            lineType)
         
        
        return img

def bounding_box(img,labels,original=None,threshold=200):
    
    if original is None:
        ret = img
    else:
        ret = original
        
    for label,size in labels:
        idx,idy = np.where(img == label)
        if len(idx) <= 1 or len(idy) <= 1 or size < threshold:
            continue
            
        
        min_x,max_x = idx.min(),idx.max()
        min_y,max_y = idy.min(),idy.max()      
        
        ret[min_x:max_x,min_y] = 255
        ret[min_x:max_x,max_y] = 255
        ret[min_x,min_y:max_y] = 255
        ret[max_x,min_y:max_y] = 255
        schwerpkt_x,schwerpkt_y = int(np.sum(idx)/len(idx)),int(np.sum(idy)/len(idy))
        ret = addText(ret,(max_y,max_x+50),"SWP: ("+str(schwerpkt_x)+","+str(schwerpkt_x)+")")
        ret[max_x:max_x+40,max_y] = 255
        

    return ret



    
def bbox(file,scale = 0.0):
    frame = Image.open(os.path.join(path,file))
    frame = max_contrast(np.array(frame))
    original = frame.copy()
    frame = binary(frame)

    if scale > 0.0:
        h,w = frame.shape[:2]
        frame = cv.resize(frame,(int(w * scale),int(h * scale)))

    label = sequentialLabeling(frame,max_dist = 3)
    frame = bounding_box(frame,label,original=original,threshold = 100)
    #frame = bounding_box(frame,label)
    #frame = mapToColor(frame,label)
    
    if scale > 0.0:
        frame = cv.resize(frame,(w,h))
    


    return frame

show_series(bbox)

In [7]:
#create_GIF(tracking,name="Schwerpunkt.gif",nbr_imgs=500)